# Flow Dashboard

Display visualizations from Github data.

## Setup

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from functools import reduce

# Date tools
from datetime import date
from dateutil.relativedelta import relativedelta, SU

# Prep Plots

# Bigger text
plt.rc('font', size=12)

def plotPie(df, title, xLabel):
  def label_point(pct, allvals):
    absolute = int(np.round(pct/100.*np.sum(allvals)))
    return "{:.1f}%\n({:d})".format(pct, absolute)

  print()
  plt.figure(figsize = (10, 5))
  plt.title(title, fontsize = 20)
  df.sum().plot(kind='pie', legend=False, 
                autopct=lambda pct: label_point(pct, df.sum()),
                pctdistance=1.16, labeldistance=.4)
  plt.ylabel(None) # Clear yLabel
  plt.xlabel(xLabel, fontsize = 16)
  plt.show()

# Date utils
def get_previous_sunday(working_date=date.today()):
    last_sunday = working_date + relativedelta(weekday=SU(-1))
    return last_sunday.strftime("%Y-%m-%d")